In [1]:
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:


def calculate_exposure_df_local(susceptibility_layer_path, aoi_path, flood_map=False, export=False, city=None):
    """
    Calculate exposure for population, nighttime light, and land cover for each susceptibility level or flood map.

    Parameters:
    susceptibility_layer_path (str): Path to the local susceptibility layer file.
    aoi_path (str): Path to the local AOI shapefile.
    flood_map (bool): Flag to indicate if susceptibility categories or flood map should be used.
    export (bool): Flag to indicate if the results should be exported to a CSV file.
    city (str): Name of the city for export file naming.

    Returns:
    pd.DataFrame: Dataframe with exposure information.
    """
    if city is None:
        city = 'city'

    # Read the susceptibility layer
    with rasterio.open(susceptibility_layer_path) as src:
        sus_layer = src.read(1)
        transform = src.transform
        crs = src.crs

    # Read the AOI
    aoi = gpd.read_file(aoi_path)
    aoi = aoi.to_crs(crs.to_string())
    
    # Define remap function for landcover
    def remapper(image):
        return np.where(np.isin(image, [1, 2, 4, 5, 7, 8, 9, 10, 11]), image, 0)

    # Load datasets (For simplicity, assuming population, nightlight, and landcover are local rasters)
    population = rasterio.open('path_to_population.tif')
    nightlight = rasterio.open('path_to_nightlight.tif')
    landcover = rasterio.open('path_to_landcover.tif')
    
    # Initialize results dictionary with descriptive land cover names
    results = {
        'Susceptibility Level': [],
        'Category': [],
        'Exposed Population': [],
        'Exposed Nighttime Light': [],
        'lulc_water': [],
        'lulc_trees': [],
        'lulc_flooded_vegetation': [],
        'lulc_crops': [],
        'lulc_built_area': [],
        'lulc_bare_ground': [],
        'lulc_snow_ice': [],
        'lulc_clouds': [],
        'lulc_rangeland': []
    }

    if flood_map:
        susceptibility_levels = [1]
        sus_layer = np.where(sus_layer > 0, 1, 0)
        category_names = ['Flooded']
    else:
        # Define quantile-based categories
        susceptibility_levels = range(1, 6)
        category_names = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']
    
    # Calculate exposure for each susceptibility level
    for level, category in zip(susceptibility_levels, category_names):
        # Mask areas that do not match the current susceptibility level
        level_mask = (sus_layer == level)
        
        # Calculate total exposed population
        exposed_population = np.sum(population.read(1)[level_mask])
        
        # Calculate total exposed nighttime light
        exposed_nightlight = np.sum(nightlight.read(1)[level_mask])
        
        # Calculate land cover area for each class
        landcover_data = landcover.read(1)
        landcover_areas = {}
        for lulc_code in [1, 2, 3, 4, 5, 6, 7, 8, 9]:
            lulc_area = np.sum(level_mask & (landcover_data == lulc_code))
            landcover_areas[lulc_code] = lulc_area
        
        # Append results to the dictionary
        results['Susceptibility Level'].append(level)
        results['Category'].append(category)
        results['Exposed Population'].append(exposed_population)
        results['Exposed Nighttime Light'].append(exposed_nightlight)
        for lulc_name, lulc_code in zip(
            ['lulc_water', 'lulc_trees', 'lulc_flooded_vegetation', 'lulc_crops',
             'lulc_built_area', 'lulc_bare_ground', 'lulc_snow_ice', 'lulc_clouds', 'lulc_rangeland'],
            [1, 2, 3, 4, 5, 6, 7, 8, 9]):
            results[lulc_name].append(landcover_areas.get(lulc_code, 0))
        
    # Convert results to dataframe
    df = pd.DataFrame(results)
    df = df.round(2)
    
    if export:
        df.to_csv(f'{city}_exposure_df.csv', index=False)
    
    return df



In [ ]:
# Example usage
sus_layer_path = 'path_to_susceptibility_layer.tif'
aoi_path = 'path_to_aoi_shapefile.shp'
exposure_df = calculate_exposure_df_local(sus_layer_path, aoi_path, flood_map=False, export=True, city='example_city')
print(exposure_df)
